In [37]:
# import tkinter as tk
# from tkinter import filedialog

import pandas as pd
import numpy as np
from functools import reduce

In [38]:
# def read_file():
#     root = tk.Tk()
#     root.withdraw()
#     root.attributes("-topmost", True)
#     files_path = filedialog.askopenfilenames(multiple=True, title='选择文件')
#     return files_path

# def loop_read_folder():
#     list_files_path = []
#     while True:
#         files_path = read_file()
#         if len(files_path) == 0:
#             break
#         else:
#             for path in files_path:
#                 list_files_path.append(path)
#     return list_files_path

# # 读取文件
# list_files_path = loop_read_folder()
# list_files_path

from Bio import Medline
'''
=========================================
助记符描述
---------------------------------------
AB摘要
CI版权信息
AD附属机构
IRAD调查员隶属关系
AID物品标识符
AU作者
FAU完整作者
CN公司作者
DCOM完成日期
DA创建日期
LR上次修订日期
DEP电子出版日期
DP出版日期
EDAT Entrez日期
GS基因符号
GN一般说明
GR授予编号
IR研究者姓名
FIR调查员全名
是ISSN
IP问题
TA期刊标题缩写
JT期刊标题
LA语言
LID位置标识符
MID手稿标识符
MHDA MeSH日期
MH MeSH术语
JID NLM唯一ID
RF参考编号
OAB其他摘要
OCI其他版权信息
OID其他ID
OT其他期限
OTO其他期限所有者
自有所有者
PG寻呼
PS个人姓名作为主题
FPS全名作为主题
PL出版地点
PHST发布历史状态
PST发布状态
PT出版物类型
PUBM发布模型
PMC PubMed中央标识符
PMID PubMed唯一标识符
RN注册号/EC号
NM物质名称
SI辅助源ID
SO来源
SFM太空飞行任务
STAT状态
SB子集
TI标题
TT翻译标题
VI卷
CON评论
CIN注释
EIN勘误表
EFR勘误表
CRI已更正并于重新发布
CRF从
PRIN部分收回
PROF部分收回
RPI于重新发布
RPF从重新发布
RIN缩回
ROF收回
UIN更新
UOF更新
SPIN总结
ORI中的原始报告
========================================='''

In [39]:
from Bio import Medline
from Bio import Entrez
import string

Entrez.email = "1219131962@qq.com"
Entrez.api_key = "6fcb01c2989c35aa1da3d9d0118abedfc409"


def get_keywords(PMID_list):
    """Thank to Dr. Mao Zhitao for his help in this function."""
    
    PMID_info = []
    for i in PMID_list:
        # 解释：db：数据库，id：文献id，rettype：返回的文献格式，retmode：返回的文献格式  #获取PMID为pmid的论文，返回medline格式的文本
        handle = Entrez.efetch(db="pubmed", id=i, rettype="medline", retmode="text")
        records = Medline.parse(handle)
        records = list(records)

        for record in records:
            title = record.get("TI", "?")  # 标题  # 解释get函数：获取字典中的值，如果没有则返回？
            abstract = record.get("AB", "?")  # 摘要
            keywords = record.get("OT", "?") # 关键词
            Journal_abbrev = record.get("TA", "?") # 期刊缩写
            Journal = record.get("JT", "?")
            country = record.get("AD", "?")[0].split(", ")[-1].strip(string.punctuation)
            pmid = record.get("PMID", "?")
            Author = record.get("FAU", "?")
            doi = record.get("SO", "?")
            date = record.get("DEP", "?")
            year = record.get("DP", "?")
            institution = record.get("AD", "?")[0]

            PMID_info.append(
                {
                    "PMID": pmid,
                    "DOI": doi,
                    "Author": Author,
                    "Title": title,
                    "Abstract": abstract,
                    "Keywords": keywords,
                    "Journal_abbrev": Journal_abbrev,
                    "Journal": Journal,
                    "Institution": institution,
                    "Country": country,
                    "Date": date,
                    "Year": year,
                }
            )
    return PMID_info

In [40]:
"""
测试文献解析代码
"""
df = pd.read_csv('./csv-Myceliopht-set.csv', sep=',').copy()

# 删除空值
df = df.dropna(subset=['PMID'])
# 删除重复值
df = df.drop_duplicates(subset=['PMID'])
df = df.reset_index(drop=True)
print("文献数量：", df.shape[0])

df


文献数量： 274


,PMID,Title,Authors,Citation,First Author,Journal/Book,Publication Year,Create Date,PMCID,NIHMS ID,DOI
0,25025273,Myceliophthora thermophila syn. Sporotrichum t...,Singh B.,Crit Rev Biotechnol. 2016;36(1):59-69. doi: 10...,Singh B,Crit Rev Biotechnol,2016,2014/7/16,NaN,NaN,10.3109/07388551.2014.923985
1,32640074,Growth kinetics of Myceliophthora thermophila ...,"Dos Santos Gomes AC, Casciatori FP, Gomes E, d...",J Appl Microbiol. 2021 Jan;130(1):90-99. doi: ...,Dos Santos Gomes AC,J Appl Microbiol,2021,2020/7/9,NaN,NaN,10.1111/jam.14774
2,24995002,Genomic insights into the fungal lignocellulol...,"Karnaouri A, Topakas E, Antonopoulou I, Christ...",Front Microbiol. 2014 Jun 18;5:281. doi: 10.33...,Karnaouri A,Front Microbiol,2014,2014/7/5,PMC4061905,NaN,10.3389/fmicb.2014.00281
3,31534479,Myceliophthora thermophila Xyr1 is predominant...,"Dos Santos Gomes AC, Falkoski D, Battaglia E, ...",Biotechnol Biofuels. 2019 Sep 16;12:220. doi: ...,Dos Santos Gomes AC,Biotechnol Biofuels,2019,2019/9/20,PMC6745793,NaN,10.1186/s13068-019-1556-y
4,35450635,Extending the diversity of Myceliophthora ther...,"Sun P, de Munnik M, van Berkel WJH, Kabel MA.",Carbohydr Polym. 2022 Jul 15;288:119373. doi: ...,Sun P,Carbohydr Polym,2022,2022/4/22,NaN,NaN,10.1016/j.carbpol.2022.119373
...,...,...,...,...,...,...,...,...,...,...,...
269,30986238,Conductive bacterial cellulose by in situ lacc...,"Shim E, Su J, Noro J, Teixeira MA, Cavaco-Paul...",PLoS One. 2019 Apr 15;14(4):e0214546. doi: 10....,Shim E,PLoS One,2019,2019/4/16,PMC6464183,NaN,10.1371/journal.pone.0214546
270,17932916,Altering the laccase functionality by in vivo ...,"Zumárraga M, Camarero S, Shleev S, Martínez-Ar...",Proteins. 2008 Apr;71(1):250-60. doi: 10.1002/...,Zumárraga M,Proteins,2008,2007/10/13,NaN,NaN,10.1002/prot.21699
271,8597577,A study of a series of recombinant fungal lacc...,"Xu F, Shin W, Brown SH, Wahleithner JA, Sundar...",Biochim Biophys Acta. 1996 Feb 8;1292(2):303-1...,Xu F,Biochim Biophys Acta,1996,1996/2/8,NaN,NaN,10.1016/0167-4838(95)00210-3
272,12817956,Spectroscopic characterization of the Leu513Hi...,"Palmer AE, Szilagyi RK, Cherry JR, Jones A, Xu...",Inorg Chem. 2003 Jun 30;42(13):4006-17. doi: 1...,Palmer AE,Inorg Chem,2003,2003/6/24,NaN,NaN,10.1021/ic026099n


In [41]:
# 设置Pubmed API访问频率
import time
from functools import reduce

request_interval = 3  # 间隔5秒

# 按每10篇文章，获取文献信息
list_PMID_info = [] 
list_interval = np.arange(0, 300, 10)
for i in list_interval:
    list_PMID = df['PMID'].tolist()[i:i+10]
    PMID_info = get_keywords(list_PMID)
    list_PMID_info.append(PMID_info)
    time.sleep(request_interval)

# 合并列表2
list_info = reduce(lambda x,y: x+y, list_PMID_info)
df_info = pd.DataFrame(list_info)
df_info['DOI'] = df_info['DOI'].str.extract(r'(10\.\d{4,9}\/[-._;()/:A-Za-z0-9]+)')
df_info['DOI'] = df_info['DOI'].str.rstrip('.')
df_info

,PMID,DOI,Author,Title,Abstract,Keywords,Journal_abbrev,Journal,Institution,Country,Date,Year
0,25025273,10.3109/07388551.2014.923985,"[Singh, Bijender]",Myceliophthora thermophila syn. Sporotrichum t...,Myceliophthora thermophila syn. Sporotrichum t...,"[Biomolecules, Myceliophthora thermophila, Spo...",Crit Rev Biotechnol,Critical reviews in biotechnology,"a Laboratory of Bioprocess Technology, Departm...",India,20140715,2016
1,32640074,10.1111/jam.14774,"[Dos Santos Gomes, A C, Casciatori, F P, Gomes...",Growth kinetics of Myceliophthora thermophila ...,AIMS: This work aimed to estimate the growth o...,"[N-acetylglucosamine, enzymes, growth kinetics...",J Appl Microbiol,Journal of applied microbiology,"Instituto de Biociencias, Letras e Ciencias Ex...",Brazil,20200729,2021 Jan
2,24995002,10.3389/fmicb.2014.00281,"[Karnaouri, Anthi, Topakas, Evangelos, Antonop...",Genomic insights into the fungal lignocellulol...,The microbial conversion of solid cellulosic b...,"[CAZy, Myceliophthora thermophila, biofuels, l...",Front Microbiol,Frontiers in microbiology,"Biotechnology Laboratory, Department of Synthe...",Sweden,20140618,2014
3,31534479,10.1186/s13068-019-1556-y,"[Dos Santos Gomes, Ana Carolina, Falkoski, Dan...",Myceliophthora thermophila Xyr1 is predominant...,BACKGROUND: Myceliophthora thermophila is a th...,"[Cellulose degradation, Myceliophthora thermop...",Biotechnol Biofuels,Biotechnology for biofuels,"1Fungal Physiology, Westerdijk Fungal Biodiver...",The Netherlands. ISNI: 0000000120346234. GRID:...,20190916,2019
4,35450635,10.1016/j.carbpol.2022.119373,"[Sun, Peicheng, de Munnik, Melanie, van Berkel...",Extending the diversity of Myceliophthora ther...,Lytic polysaccharide monooxygenases (LPMOs) pl...,"[Active site segment, LPMOs, Lignocellulose, M...",Carbohydr Polym,Carbohydrate polymers,"Laboratory of Food Chemistry, Wageningen Unive...",the Netherlands. Electronic address: peicheng....,20220318,2022 Jul 15
...,...,...,...,...,...,...,...,...,...,...,...,...
269,30986238,10.1371/journal.pone.0214546,"[Shim, Euijin, Su, Jing, Noro, Jennifer, Teixe...",Conductive bacterial cellulose by in situ lacc...,Conductive and colored bacterial cellulose (BC...,?,PLoS One,PloS one,"Department of Clothing and Textiles, Sookmyung...",South Korea,20190415,2019
270,17932916,10.1002/prot.21699,"[Zumarraga, Miren, Camarero, Susana, Shleev, S...",Altering the laccase functionality by in vivo ...,The generation of diversity for directed prote...,?,Proteins,Proteins,"Instituto de Catalisis y Petroleoquimica, CSIC...",Spain,?,2008 Apr
271,8597577,10.1016/0167-4838(95)00210-3,"[Xu, F, Shin, W, Brown, S H, Wahleithner, J A,...",A study of a series of recombinant fungal lacc...,A series of fungal laccases (Polyporus pinsitu...,?,Biochim Biophys Acta,Biochimica et biophysica acta,"Novo Nordisk Biotech, Davis, CA 95616 USA.",CA 95616 USA,?,1996 Feb 8
272,12817956,10.1021/ic026099n,"[Palmer, Amy E, Szilagyi, Robert K, Cherry, Jo...",Spectroscopic characterization of the Leu513Hi...,"A variety of spectroscopic techniques, combine...",?,Inorg Chem,Inorganic chemistry,"Department of Chemistry, Stanford University, ...",USA,?,2003 Jun 30


In [31]:
# 保存为csv文件
df_info.to_csv('./df_info_输出示例.csv', index=False, sep=',', encoding='utf_8_sig')

In [32]:
# df_info的PMID列有重复值，需要删除
df_info = df_info.drop_duplicates(subset=['PMID'])
df_info = df_info.reset_index(drop=True)
df_info

,PMID,DOI,Author,Title,Abstract,Keywords,Journal_abbrev,Journal,Institution,Country,Date,Year
0,35858815,10.2323/jgam.2022.06.002,"[Wang, Ruijie, Arioka, Manabu]",Glucuronoyl esterase facilitates biomass degra...,Glucuronoyl esterase (GE) is a promising agent...,"[Neurospora crassa, cellulase, glucuronoyl est...",J Gen Appl Microbiol,The Journal of general and applied microbiology,"Department of Biotechnology, The University of...",The University of Tokyo,20220721,2023 Mar 6
1,36481248,10.1016/j.fgb.2022.103763,"[Wang, Ziyan, Bartholomai, Bradley M, Loros, J...",Optimized fluorescent proteins for 4-color and...,Fungal cells are quite unique among life in th...,?,Fungal Genet Biol,Fungal genetics and biology : FG & B,"Geisel School of Medicine at Dartmouth, Depart...",USA,20221205,2023 Jan
2,37220856,10.1016/j.jbc.2023.104850,"[Wang, Bin, Dunlap, Jay C]",Domains required for the interaction of the ce...,In the negative feedback loop composing the Ne...,"[FFC, FRH, FRQ, Neurospora, WC-1, WC-2, WCC, a...",J Biol Chem,The Journal of biological chemistry,"Department of Molecular and Systems Biology, G...",USA. Electronic address: Bin.Wang@Dartmouth.edu,20230521,2023 Jul
3,37313736,10.1093/genetics/iyad112,"[Mela, Alexander P, Glass, N Louise]",Permissiveness and competition within and betw...,A multinucleate syncytium is a common growth f...,"[Neurospora crassa, allorecognition, cell fusi...",Genetics,Genetics,"The Plant and Microbial Biology Department, Un...",USA,?,2023 Aug 9
4,36744948,10.1128/mbio.03291-22,"[Tabilo-Agurto, Cyndi, Del Rio-Pinilla, Veroni...",Developing a Temperature-Inducible Transcripti...,"Heat shock protein (HSP)-encoding genes (hsp),...","[HSP, Neurospora, Neurospora crassa, heat shoc...",mBio,mBio,Departamento de Genetica Molecular y Microbiol...,Chile,20230206,2023 Feb 28
...,...,...,...,...,...,...,...,...,...,...,...,...
318,30986238,10.1371/journal.pone.0214546,"[Shim, Euijin, Su, Jing, Noro, Jennifer, Teixe...",Conductive bacterial cellulose by in situ lacc...,Conductive and colored bacterial cellulose (BC...,?,PLoS One,PloS one,"Department of Clothing and Textiles, Sookmyung...",South Korea,20190415,2019
319,17932916,10.1002/prot.21699,"[Zumarraga, Miren, Camarero, Susana, Shleev, S...",Altering the laccase functionality by in vivo ...,The generation of diversity for directed prote...,?,Proteins,Proteins,"Instituto de Catalisis y Petroleoquimica, CSIC...",Spain,?,2008 Apr
320,8597577,10.1016/0167-4838(95)00210-3,"[Xu, F, Shin, W, Brown, S H, Wahleithner, J A,...",A study of a series of recombinant fungal lacc...,A series of fungal laccases (Polyporus pinsitu...,?,Biochim Biophys Acta,Biochimica et biophysica acta,"Novo Nordisk Biotech, Davis, CA 95616 USA.",CA 95616 USA,?,1996 Feb 8
321,12817956,10.1021/ic026099n,"[Palmer, Amy E, Szilagyi, Robert K, Cherry, Jo...",Spectroscopic characterization of the Leu513Hi...,"A variety of spectroscopic techniques, combine...",?,Inorg Chem,Inorganic chemistry,"Department of Chemistry, Stanford University, ...",USA,?,2003 Jun 30


In [33]:
# 保存为csv文件
df_info.to_csv('./df_info_输出示例-2.csv', index=False, sep=',', encoding='utf_8_sig')

In [28]:
# 根据df_info['Journal']的信息，绘制柱状图
import plotly.express as px
import plotly.graph_objects as go

df_info_gro_jour = df_info.groupby('Journal').count().sort_values(by='PMID', ascending=False).iloc[:50, :]
fig = px.scatter(df_info_gro_jour, x=df_info_gro_jour.index, y=df_info_gro_jour['PMID'], title='Journals', labels={'x':'Journal', 'y':'Count'},)
fig.update_layout(height=1000, xaxis_tickangle=45, template='plotly', font_size=20)
fig.show()